# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [14]:
from azureml.core import Workspace, Experiment, Environment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice
import os

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [15]:
ws = Workspace.from_config()
experiment_name = 'hyperdrive_exp'

experiment=Experiment(ws, experiment_name)

In [16]:
compute_target = ComputeTarget(workspace=ws, name="cpu-sbinstance")

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [18]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=1, slack_factor=0.2, delay_evaluation=5)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({"--C": choice(1,2,3,4,5), "--max_iter": choice(80,100,120,150,170,200)})

#TODO: Create your estimator and hyperdrive config
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')
current_dir = os.getcwd()
arguments = {"--C": 1.0, "--max_iter": 100} 
estimator = SKLearn(source_directory=current_dir, compute_target=compute_target, entry_script='train.py', script_params=arguments,environment_definition=sklearn_env)
hyperdrive_run_config = HyperDriveConfig(estimator=estimator, policy=early_termination_policy, primary_metric_name="Accuracy",
                                          hyperparameter_sampling=param_sampling,
                                         max_total_runs=20,
                                          primary_metric_goal=PrimaryMetricGoal.MAXIMIZE)

In [19]:
#TODO: Submit your experiment
hyperDrive_run = experiment.submit(hyperdrive_run_config)

The same input parameter(s) are specified in estimator/run_config script params and HyperDrive parameter space. HyperDrive parameter space definition will override these duplicate entries. ['--C', '--max_iter'] is the list of overridden parameter(s).


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [20]:
RunDetails(hyperDrive_run).show()
hyperDrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_b7f558a1-4f3d-423c-9b61-e766dff7b1f1
Web View: https://ml.azure.com/runs/HD_b7f558a1-4f3d-423c-9b61-e766dff7b1f1?wsid=/subscriptions/6b4af8be-9931-443e-90f6-c4c34a1f9737/resourcegroups/aml-quickstarts-252188/workspaces/quick-starts-ws-252188&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2024-02-09T06:30:58.067505][GENERATOR][INFO]Trying to sample '20' jobs from the hyperparameter space
[2024-02-09T06:30:58.6662673Z][SCHEDULER][INFO]Scheduling job, id='HD_b7f558a1-4f3d-423c-9b61-e766dff7b1f1_0' 
[2024-02-09T06:30:58.7805365Z][SCHEDULER][INFO]Scheduling job, id='HD_b7f558a1-4f3d-423c-9b61-e766dff7b1f1_1' 
[2024-02-09T06:30:58.8757888Z][SCHEDULER][INFO]Scheduling job, id='HD_b7f558a1-4f3d-423c-9b61-e766dff7b1f1_2' 
[2024-02-09T06:30:59.1219298Z][SCHEDULER][INFO]Scheduling job, id='HD_b7f558a1-4f3d-423c-9b61-e766dff7b1f1_3' 
[2024-02-09T06:30:59.2560074Z][SCHEDULER][INFO]Scheduling job, id='HD_b7f558a1-4f3d-423c-9b61-e766dff7b1f1_4' 
[2024-02-0

{'runId': 'HD_b7f558a1-4f3d-423c-9b61-e766dff7b1f1',
 'target': 'cpu-sbinstance',
 'status': 'Completed',
 'startTimeUtc': '2024-02-09T06:30:57.326577Z',
 'endTimeUtc': '2024-02-09T06:45:25.232101Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '2e9d2195-2b7a-4a4c-b14e-7db5353406df',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1040-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.51.0',
  'space_size': '30',
  'best_child_run_id': 'HD_b7f558a1-4f3d-423c-9b61-e766dff7b1f1_7',
  'score': '0.9066666666666666',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_b7f558a1-4f3d-423c-9b61-e766dff7b1f1_7'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClie

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [21]:
best_run = hyperDrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n Regularization Strength:',best_run_metrics['Regularization Strength:'])
print('\n Max Iterations:',best_run_metrics['Max iterations:'])

Best Run Id:  HD_b7f558a1-4f3d-423c-9b61-e766dff7b1f1_7

 Accuracy: 0.9066666666666666

 Regularization Strength: 2.0

 Max Iterations: 120


In [26]:
#TODO: Save the best model
best_model = best_run.register_model(model_path='logs/', model_name='hyper_drive_best_model',
                        properties={'Accuracy': best_run_metrics['Accuracy']})

print(best_model)

Model(workspace=Workspace.create(name='quick-starts-ws-252188', subscription_id='6b4af8be-9931-443e-90f6-c4c34a1f9737', resource_group='aml-quickstarts-252188'), name=hyper_drive_best_model, id=hyper_drive_best_model:1, version=1, tags={}, properties={'Accuracy': '0.9066666666666666'})


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

